# 🐟 Filogenia molecular de *Leporinus* usando COI

Notebook para ejecutar en **Google Colab** (o cualquier entorno Jupyter) el flujo descrito en el README:

- Cargar el archivo `ANOS_COI.fasta`
- Revisar calidad básica de las secuencias
- Alinear con **MAFFT**
- Inferir un árbol de Máxima Verosimilitud con **IQ-TREE**
- Visualizar el árbol

---

## 1️⃣ Instrucciones iniciales

1. Sube el archivo **`ANOS_COI.fasta`** al entorno de Colab.
   - En Colab, haz clic en el ícono de carpeta (barra lateral izquierda).
   - Haz clic en el ícono de **subir (upload)**.
   - Selecciona tu archivo `ANOS_COI.fasta` desde tu computadora.
   - El archivo quedará en la ruta de trabajo actual, típicamente `/content/ANOS_COI.fasta`.

2. Ejecuta las celdas **en orden**, de arriba hacia abajo.

A partir de aquí, el notebook asumirá que el archivo se llama exactamente `ANOS_COI.fasta` y está en el directorio de trabajo actual.

In [ ]:
#@title Ver archivos presentes en el entorno
!pwd
!ls

## 2️⃣ Instalación de dependencias

En esta celda instalamos:

- **MAFFT** → para el alineamiento múltiple
- **IQ-TREE** → para la inferencia filogenética por Máxima Verosimilitud
- **Biopython** → para manipular archivos FASTA y árboles
- **matplotlib** y **pandas** → para tablas y gráficos de calidad

Esto puede tardar algunos minutos la primera vez.

In [ ]:
#@title Instalar MAFFT, IQ-TREE y librerías de Python
%%bash
set -e
apt-get update -qq
apt-get install -y mafft iqtree -qq
python -m pip install --quiet biopython matplotlib pandas

## 3️⃣ Carga y revisión básica del archivo FASTA

En esta sección:
- Leemos `ANOS_COI.fasta`
- Contamos cuántas secuencias hay
- Calculamos longitudes mínima, máxima y promedio
- Contamos cuántas **N** tiene cada secuencia
- Generamos una tabla resumen y gráficos simples

In [ ]:
#@title Leer FASTA y hacer control de calidad básico
from Bio import SeqIO
import pandas as pd
import matplotlib.pyplot as plt

fasta_path = 'ANOS_COI.fasta'  # Cambia esto si tu archivo tiene otro nombre

records = list(SeqIO.parse(fasta_path, 'fasta'))
print(f"Número total de secuencias cargadas: {len(records)}")

data = []
for rec in records:
    seq_str = str(rec.seq).upper()
    length = len(seq_str)
    n_count = seq_str.count('N')
    data.append({
        'id': rec.id,
        'longitud': length,
        'Ns': n_count
    })

df = pd.DataFrame(data)
display(df.head())
print('\nResumen de longitudes:')
display(df['longitud'].describe())
print('\nResumen de Ns:')
display(df['Ns'].describe())

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].hist(df['longitud'], bins=20)
axes[0].set_title('Distribución de longitudes')
axes[0].set_xlabel('Longitud (bp)')
axes[0].set_ylabel('Frecuencia')

axes[1].hist(df['Ns'], bins=20)
axes[1].set_title('Distribución de Ns por secuencia')
axes[1].set_xlabel('Número de Ns')
axes[1].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

## 4️⃣ Alineamiento de las secuencias con MAFFT

En esta celda:
- Ejecutamos **MAFFT** con el modo automático (`--auto`)
- Usamos 4 hilos (`--thread 4`, puedes ajustar este valor)
- Guardamos el alineamiento en `ANOS_COI_aligned.fasta`

⚠️ Nota: Si quieres usar otro nombre de archivo, cámbialo tanto aquí como en las celdas siguientes.

In [ ]:
#@title Ejecutar MAFFT para alinear las secuencias
%%bash
set -e
echo 'Iniciando alineamiento con MAFFT...'
mafft --auto --thread 4 ANOS_COI.fasta > ANOS_COI_aligned.fasta
echo 'Alineamiento completado. Archivo: ANOS_COI_aligned.fasta'

### Verificación rápida del alineamiento

Aquí cargamos el archivo alineado y verificamos:
- Longitud del alineamiento (número de columnas)
- Algunas secuencias de ejemplo.

In [ ]:
#@title Cargar y revisar el alineamiento
from Bio import AlignIO

alignment = AlignIO.read('ANOS_COI_aligned.fasta', 'fasta')
print(f"Número de secuencias en el alineamiento: {len(alignment)}")
print(f"Longitud del alineamiento (columnas): {alignment.get_alignment_length()}")

print('\nPrimeras 2 secuencias (primeros 80 caracteres alineados):')
for rec in alignment[:2]:
    print(rec.id)
    print(str(rec.seq)[:80])
    print('...')

## 5️⃣ Inferencia del árbol filogenético con IQ-TREE

En esta celda corremos IQ-TREE con:

- `-s ANOS_COI_aligned.fasta` → alineamiento de entrada
- `-m MFP` → ModelFinder (selección automática del mejor modelo de sustitución)
- `-bb 1000` → 1000 réplicas de **bootstrap ultrarrápido**
- `-nt AUTO` → detección automática del número de hilos

Esto generará varios archivos. El árbol principal estará en `ANOS_COI_aligned.fasta.treefile`.

In [ ]:
#@title Ejecutar IQ-TREE (Máxima Verosimilitud + bootstrap)
%%bash
set -e
echo 'Iniciando IQ-TREE...'
iqtree -s ANOS_COI_aligned.fasta -m MFP -bb 1000 -nt AUTO
echo 'IQ-TREE finalizado.'

### Archivos generados por IQ-TREE

Listamos los archivos en el directorio actual para identificar la salida de IQ-TREE (incluyendo el `.treefile`).

In [ ]:
#@title Listar archivos generados
!ls

## 6️⃣ Visualización del árbol con Biopython (Phylo)

En esta sección:
- Leemos el archivo `.treefile` generado por IQ-TREE
- Dibujamos el árbol usando `Bio.Phylo`

Puedes ajustar el tamaño de la figura para ver mejor las etiquetas.

In [ ]:
#@title Cargar y dibujar el árbol de IQ-TREE
from Bio import Phylo
import matplotlib.pyplot as plt

treefile = 'ANOS_COI_aligned.fasta.treefile'

tree = Phylo.read(treefile, 'newick')
print(tree)

fig = plt.figure(figsize=(10, 20))  # ajusta el tamaño según la cantidad de secuencias
axes = fig.add_subplot(1, 1, 1)
Phylo.draw(tree, do_show=True, axes=axes)

## 7️⃣ (Opcional) Árbol en formato ASCII

Si el gráfico no se ve bien o hay muchas secuencias, puedes usar la representación en texto (ASCII) para una vista rápida.

In [ ]:
#@title Mostrar árbol en formato ASCII (texto)
from Bio import Phylo

treefile = 'ANOS_COI_aligned.fasta.treefile'
tree = Phylo.read(treefile, 'newick')
Phylo.draw_ascii(tree)

## ✅ Resumen

En este notebook hemos:

1. Cargado el archivo `ANOS_COI.fasta` con secuencias COI de *Leporinus*.
2. Realizado un control de calidad básico (longitud, Ns).
3. Alineado las secuencias con **MAFFT**.
4. Inferido un árbol de Máxima Verosimilitud con **IQ-TREE** (ModelFinder + bootstrap).
5. Visualizado el árbol con **Biopython Phylo**.

Esto confirma que el flujo descrito en tu README se puede ejecutar completamente en Python (vía Colab), utilizando herramientas estándar de filogenia.

Puedes adaptar este notebook para tu repositorio de GitHub, o dividir sus pasos en scripts `.py` si deseas un pipeline más modular.